# Pandas MultiIndex WorkOut

# When should you use one?

1. When a single column’s value isn’t enough to uniquely identify a row (e.g. multiple records on the same date means date alone isn’t a good index).
2. When data is logically hierarchical - meaning that it has multiple dimensions or “levels.”

Besides structure, multiindexes offer relatively easy in-memory retreival of complex data.

# Grocery Shop

For this exercise we will work with a realistic case of when multiindex DataFrames can come in handy - a grocer's retail transactions.

In [19]:
from typing import Any
import numpy as np
import pandas as pd

display(f"Pandas version: {pd.__version__}")

'Pandas version: 0.25.3'

In [20]:
df = pd.read_csv('data.csv', parse_dates=['Date'])
df.sample(10)
df.shape


(132, 8)

Each row of data represents a sale of an item, but how do we set our index meaningfully? A numeric ID for each transaction would be fine, but it wouldn't tell us terribly much. Even the date of a transaction isn't useful by itself, since it's common for such a company to have many transactions on a given date (even at the same time). 

Instead, we will have to look at a combination of the available metadata to create a unique and meaningful index. At first glance, a combination of the date, the store, and the hierarchy of each item (Category > Subcategory > UPC) look like they might be enough to get us a unique index.

# Setting and Manipulating MultiIndexes



So let's take a look at how we can create our multiindex from our regular ol' DataFrame. We'll walk through the basics of setting, reordering, and resetting indexes, along with some useful tips/tricks. Then we can begin investigating our transaction data to learn about our sales and trends. Let's create a multiindex with 'Date', 'Store', 'Category', 'Subcategory' and 'Description'

In [21]:

# we give a list of column names instead of a single string column name

df.set_index(['Date', 'Store', 'Category', 'Subcategory', 'Description'], inplace=True)
df.head(3)

UPC EAN  \
Date       Store   Category Subcategory Description                                         
2018-07-11 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack  736920111112   
           Store 2 Beer     Ales        Goose Island - Honkers Ale - 6 Pack  736920111112   
2018-07-10 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack  736920111112   

                                                                             Dollars  \
Date       Store   Category Subcategory Description                                    
2018-07-11 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack    35.68   
           Store 2 Beer     Ales        Goose Island - Honkers Ale - 6 Pack   166.74   
2018-07-10 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack    67.36   

                                                                             Units  
Date       Store   Category Subcategory Description                                 
2018-07-11 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack      4  
           Store 2 Beer     Ales        Goose Island - Honkers Ale - 6 Pack     14  
2018-07-10 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack      8

Add the 'UPC EAN' column to our index.

In [22]:
# We can append a column to our existing index
df.set_index('UPC EAN', append=True, inplace=True)
df.head(3)

Dollars  \
Date       Store   Category Subcategory Description                         UPC EAN                 
2018-07-11 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack 736920111112    35.68   
           Store 2 Beer     Ales        Goose Island - Honkers Ale - 6 Pack 736920111112   166.74   
2018-07-10 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack 736920111112    67.36   

                                                                                          Units  
Date       Store   Category Subcategory Description                         UPC EAN              
2018-07-11 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack 736920111112      4  
           Store 2 Beer     Ales        Goose Island - Honkers Ale - 6 Pack 736920111112     14  
2018-07-10 Store 3 Beer     Ales        Goose Island - Honkers Ale - 6 Pack 736920111112      8

That's almost right, but we'd actually like 'Description' to show up after 'UPC EAN'. We can do this using 2 differente ways (reorder and swaplevel)

In [23]:
# Option 1 is the generalized solution to reorder the index levels
# Note: We're not making an inplace change in this cell,
#       but it's worth noting that this method doesn't have an inplace parameter.
df.reorder_levels(order=['Date', 'Store', 'Category', 'Subcategory', 'UPC EAN', 'Description']).head(3)

Dollars  \
Date       Store   Category Subcategory UPC EAN      Description                                    
2018-07-11 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack    35.68   
           Store 2 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack   166.74   
2018-07-10 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack    67.36   

                                                                                          Units  
Date       Store   Category Subcategory UPC EAN      Description                                 
2018-07-11 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack      4  
           Store 2 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack     14  
2018-07-10 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack      8

In [24]:
# Option 2 just switches two index levels (a more common need than you'd think)
# Note: This time we're doing an inplace change, but there's no parameter for this method either.
df = df.swaplevel('Description', 'UPC EAN')
df.head(3)

Dollars  \
Date       Store   Category Subcategory UPC EAN      Description                                    
2018-07-11 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack    35.68   
           Store 2 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack   166.74   
2018-07-10 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack    67.36   

                                                                                          Units  
Date       Store   Category Subcategory UPC EAN      Description                                 
2018-07-11 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack      4  
           Store 2 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack     14  
2018-07-10 Store 3 Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack      8

Just when we thought we were done, it turns our we forgot to add the highest level of the product hierarchy - the Department - not just to our index, but to our DataFrame altogether. Luckily all of our records belong in the same Department. Let's include a new department column:

In [27]:
# A handy function to keep around for projects
def add_constant_index_level(df: pd.DataFrame, value: Any, level_name: str):
    """Add a new level to an existing index where every row has the same, given value.
    
    Args:
        df: Any existing pd.DataFrame.
        value: Value to be placed in every row of the new index level.
        level_name: Title of the new index level.
    
    Returns:
        df with an additional, prepended index level.
    """
    return pd.concat([df], keys=[value], names=[level_name])

df = add_constant_index_level(df, "Booooze", "Department")
df = df.reorder_levels(order=['Date', 'Store', 'Department', 'Category', 'Subcategory', 'UPC EAN', 'Description'])
df.head(3)

# # If we wanted to later drop that level
# df.index = df.index.droplevel(level='Department')
# df.head(3)

Dollars  \
Date       Store   Department Category Subcategory UPC EAN      Description                                    
2018-07-11 Store 3 Booooze    Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack    35.68   
           Store 2 Booooze    Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack   166.74   
2018-07-10 Store 3 Booooze    Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack    67.36   

                                                                                                     Units  
Date       Store   Department Category Subcategory UPC EAN      Description                                 
2018-07-11 Store 3 Booooze    Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack      4  
           Store 2 Booooze    Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack     14  
2018-07-10 Store 3 Booooze    Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack      8

Now that our index is set the way we want it, what if we want to interact with those index levels? Here are a few helpful code snippets:

In [18]:
# checking out their unique values, for a single level 
df.index.get_level_values('Subcategory').unique()

# checking out their unique values, for combinations of multiple levels
# See answer at https://stackoverflow.com/questions/39080555/pandas-get-level-values-for-multiple-columns

Index(['Ales', 'Lagers', 'Stouts', 'Malts', 'Red', 'White', 'Rose', 'Liqour',
       'Liquor'],
      dtype='object', name='Subcategory')

Note the typo in "Liquor" above. Good thing we checked out unique values! Maybe someone can submit a pull request to fix this for me :)

# Understanding the MultiIndex Object

In [9]:
df.index

MultiIndex([('2018-07-11', 'Store 3', 'Booooze',    'Beer',   'Ales', ...),
            ('2018-07-11', 'Store 2', 'Booooze',    'Beer',   'Ales', ...),
            ('2018-07-10', 'Store 3', 'Booooze',    'Beer',   'Ales', ...),
            ('2018-07-10', 'Store 2', 'Booooze',    'Beer',   'Ales', ...),
            ('2018-07-12', 'Store 3', 'Booooze',    'Beer',   'Ales', ...),
            ('2018-07-12', 'Store 2', 'Booooze',    'Beer',   'Ales', ...),
            ('2018-07-14', 'Store 3', 'Booooze',    'Beer',   'Ales', ...),
            ('2018-07-14', 'Store 2', 'Booooze',    'Beer',   'Ales', ...),
            ('2018-07-10', 'Store 2', 'Booooze',    'Beer',   'Ales', ...),
            ('2018-07-10', 'Store 3', 'Booooze',    'Beer',   'Ales', ...),
            ...
            ('2018-07-13', 'Store 2', 'Booooze', 'Alcohol', 'Liquor', ...),
            ('2018-07-13', 'Store 3', 'Booooze', 'Alcohol', 'Liquor', ...),
            ('2018-07-11', 'Store 2', 'Booooze', 'Alcohol', 'Liquor', ..

Well that's gross looking...but don't be scared - it's actually not that hard to understand.

**'levels'** is a list of lists, where each sublist represents all possible values in that index level. In other words, the 'levels' parameter reflects all possible unique values by level. For example, our first index level ('Date') has the possible values \['2018-07-10', '2018-07-11', '2018-07-12', ...\].

* **Important Note:** When talking about a multiindex DataFrame (not the parameter for the MultiIndex object), we talk about the "levels" as the index "columns." For example, the 'levels' of our df in a more general sense are 'Date', 'Store', 'Department', etc. Levels in this sense (and elsewhere in code) can also be referenced by number (e.g. 'Date' = 0 \[read as 'level 0'\], 'Store' = 1, 'Department' = 2, etc.).

**'labels'** is also a list of lists, but here each sublist reflects all of the values that appear in the row of that index. In other words, each sublist in our labels is of the same length as the entire dataframe, and the value of each row is one of the possible values defined in our associated level (above). Looking again at our first index level ('Date'), we see values like \[1, 1, 0, 0, 2, 2, 4, 4 ...\]. There are just an enumerated representation of the options defined in our level, so 0 = '2018-07-10', 1 = '2018-07-11', 2 = '2018-07-12', 3 = '2018-07-13', etc.

**'names'** is a list of the actual titles of each index level, in order of appearance from left to right.

With that fresh understanding of the 'anatomy' of a MultiIndex, we can look at...

# MultiIndex Columns (Multiple Column Levels)

For a different view we can also create hierarchical column levels. We'll do this by introducting a new method: unstack(). This function "pivots" an index level to a new level of column labels whose inner-most level consists of the pivoted index labels. **Stack/unstack is one of the biggest reasons to use a MultiIndex, so it's work supplementing the examples here by checking out the [official docs on reshaping](https://pandas.pydata.org/pandas-docs/stable/reshaping.html#reshaping-by-stacking-and-unstacking).**

With this new technique we can start actually investigating our data. For example, let's say we want to more easily compare sales of a product by store by day, we can unstack our 'Store' index level:

In [10]:
multi_col_lvl_df = df.unstack('Store')
multi_col_lvl_df.sample(10)

Dollars  \
Store                                                                                               Store 1   
Date       Department Category Subcategory UPC EAN      Description                                           
2018-07-15 Booooze    Wine     Red         492130008399 Yellow Tail - Merlot - 750 mL Bottle          54.27   
2018-07-12 Booooze    Beer     Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack      NaN   
2018-07-13 Booooze    Wine     Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle   131.60   
2018-07-16 Booooze    Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack          NaN   
2018-07-11 Booooze    Beer     Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack      NaN   
2018-07-12 Booooze    Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack             NaN   
2018-07-11 Booooze    Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle             50.40   
                      Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle     NaN   
                      Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack             NaN   
2018-07-12 Booooze    Wine     Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle    93.94   

                                                                                                             \
Store                                                                                               Store 2   
Date       Department Category Subcategory UPC EAN      Description                                           
2018-07-15 Booooze    Wine     Red         492130008399 Yellow Tail - Merlot - 750 mL Bottle            NaN   
2018-07-12 Booooze    Beer     Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack   116.71   
2018-07-13 Booooze    Wine     Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle   156.53   
2018-07-16 Booooze    Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack        99.40   
2018-07-11 Booooze    Beer     Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack    56.14   
2018-07-12 Booooze    Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack           37.75   
2018-07-11 Booooze    Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle               NaN   
                      Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle  248.80   
                      Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack          166.74   
2018-07-12 Booooze    Wine     Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle    83.82   

                                                                                                             \
Store                                                                                               Store 3   
Date       Department Category Subcategory UPC EAN      Description                                           
2018-07-15 Booooze    Wine     Red         492130008399 Yellow Tail - Merlot - 750 mL Bottle          58.20   
2018-07-12 Booooze    Beer     Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack   109.34   
2018-07-13 Booooze    Wine     Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle      NaN   
2018-07-16 Booooze    Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack        46.65   
2018-07-11 Booooze    Beer     Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack   143.52   
2018-07-12 Booooze    Beer     Ales        736920111112 Goose Island - Honkers Ale - 6 Pack           78.30   
2018-07-11 Booooze    Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle             47.25   
                      Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle  2

# Multiindex Math

Math operations are as nearly as easy with multiindex DataFrames as with regular ones, but a whole lot more powerful. Let's say we want to calculate dollars per unit for each of our stores. That's as simple as:

In [44]:
dollars_per_unit = multi_col_lvl_df['Dollars'] / multi_col_lvl_df['Units']
dollars_per_unit.sample(10)

Store                                                                                                      Store 1  \
Date       Department Category Subcategory UPC EAN      Description                                                  
2018-07-12 Booooze    Wine     White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle            NaN   
2018-07-16 Booooze    Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                     9.01   
2018-07-12 Booooze    Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                     7.53   
2018-07-16 Booooze    Beer     Ales        702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack      NaN   
2018-07-11 Booooze    Alcohol  Liquor      674545000001 Don Julio - Tequila Blanco - 750 mL Bottle             NaN   
2018-07-15 Booooze    Alcohol  Liquor      674545000001 Don Julio - Tequila Blanco - 750 mL Bottle             NaN   
2018-07-12 Booooze    Alcohol  Liquor      674545000001 Don Julio - Tequila Blanco - 750 mL Bottle             NaN   
                      Wine     Red         492130008399 Yellow Tail - Merlot - 750 mL Bottle                  7.93   
2018-07-14 Booooze    Beer     Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack             NaN   
2018-07-16 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle            NaN   

Store                                                                                                      Store 2  \
Date       Department Category Subcategory UPC EAN      Description                                                  
2018-07-12 Booooze    Wine     White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle          21.86   
2018-07-16 Booooze    Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                      NaN   
2018-07-12 Booooze    Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                      NaN   
2018-07-16 Booooze    Beer     Ales        702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack     9.71   
2018-07-11 Booooze    Alcohol  Liquor      674545000001 Don Julio - Tequila Blanco - 750 mL Bottle           33.81   
2018-07-15 Booooze    Alcohol  Liquor      674545000001 Don Julio - Tequila Blanco - 750 mL Bottle           29.50   
2018-07-12 Booooze    Alcohol  Liquor      674545000001 Don Julio - Tequila Blanco - 750 mL Bottle           34.24   
                      Wine     Red         492130008399 Yellow Tail - Merlot - 750 mL Bottle                   NaN   
2018-07-14 Booooze    Beer     Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack            9.68   
2018-07-16 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle          24.56   

Store                                                                                                      Store 3  
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-12 Booooze    Wine     White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle          21.40  
2018-07-16 Booooze    Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                     8.27  
2018-07-12 Booooze    Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                     5.33  
2018-07-16 Booooze    Beer     Ales        702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack    14.36  
2018-07-11 Booooze    Alcohol  Liquor      674545000001 Don Julio - Tequila Blanco - 750 mL Bottle           25.51  
2018-07-15 Booooze    Alcohol  Liquor      674545000001 Don Julio - Tequila Blanco - 750 mL Bottle           30.75  
2018-07-12 Booooze    Alcohol  Liquor      674545000001 Don Julio - Tequila Blanco - 750 mL Bottle           29.84  
                      Wine     Red         492130008399 Yellow Tail - Merlot - 750 mL Bottle                  7.5

We just operated on all the pairwise sub column levels at the same time! Now we'd like this answer as columns back in our original DataFrame:

In [45]:
# Add a column level for our new measure
dollars_per_unit.columns = pd.MultiIndex.from_product([['Dollars per Unit'], dollars_per_unit.columns])

# Concat it with our original data
pd.concat([multi_col_lvl_df, dollars_per_unit], axis='columns').head(3)

Dollars  \
Store                                                                                                     Store 1   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            NaN   

                                                                                                                   \
Store                                                                                                     Store 2   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        231.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  172.00   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         168.06   

                                                                                                                   \
Store                                                                                                     Store 3   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        128.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  278.80   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         204.12   

                                                                                                            Units  \
Store                                                                                                     Store 1   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            NaN   

                                                                                                                   \
Store                                                                                                     Store 2   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           9.0   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     5.0   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            6.0   

                                                                                                                   \
Store                                                                                                     Store 3   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           5.0   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     8.0   
      

We can similarly apply functions to our multiindex DataFrame:

In [46]:
# Change our units to 000s for funsies
multi_col_lvl_df.applymap(lambda x: np.nan if np.isnan(x) else str(round(x/1000, 2)) + "k").head(10)

Dollars  \
Store                                                                                                     Store 1   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            NaN   
                      Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack                NaN   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack     NaN   
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                   NaN   
                               Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack               0.14k   
                               Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack            NaN   
                      Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                   0.06k   
                                           492130008399 Yellow Tail - Merlot - 750 mL Bottle                0.05k   

                                                                                                                   \
Store                                                                                                     Store 2   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle         0.23k   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle   0.17k   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle          0.17k   
                      Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack              0.08k   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack   0.09k   
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                 0.08k   
                               Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack                 NaN   
                               Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack          0.08k   
                      Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                     NaN   
                                           492130008399 Yellow Tail - Merlot - 750 mL Bottle                  NaN   

                                                                                                                   \
Store                                                                                                     Store 3   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle         0.13k   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle   0.28k   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle           0.2k   
                      Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack              0.06k   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack   0.14k   
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                 0.07k   
         

# Sorting

Before we start slicing and filtering, it's important to sort out data to drastically improve efficiency. In some cases pandas will begin sorting for you by default in the latest versions, but other times you'll get an ugly and confusing lexsort warning (referencing the default lexographic sorting order).

Sorting indexes works the exactly the same way for multiindex DataFrames as with regular DataFrames, but with some extra parameters to decide on:

In [47]:
# By default sort_index will sort all levels of the index,
# first sorting the first index level, then secondarily sorting the second index level, and so on...
sort1 = multi_col_lvl_df.copy()
sort1.sort_index(inplace=True)
print("\nSort by Date, then Department, then Category, etc.:")
display(sort1.head(10))

# ...but you can choose to starting from a different level...
sort2 = multi_col_lvl_df.copy()
sort2.sort_index(level='Category', inplace=True)
print("\nSort by Category, then Subcategory, then UPC EAN, then Description, then Date, then Deparment:")
display(sort2.head(10))

# ...or even to only sort on specifc levels.
sort3 = multi_col_lvl_df.copy()
sort3.sort_index(level=['Category', 'Date'], sort_remaining=False, inplace=True)
print("\nSort by Category, then Date only:")
display(sort3.head(10))


Sort by Date, then Department, then Category, etc.:


Dollars  \
Store                                                                                                     Store 1   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            NaN   
                      Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack                NaN   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack     NaN   
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                   NaN   
                               Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack              137.67   
                               Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack            NaN   
                      Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                   56.88   
                                           492130008399 Yellow Tail - Merlot - 750 mL Bottle                46.30   

                                                                                                                   \
Store                                                                                                     Store 2   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        231.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  172.00   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         168.06   
                      Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack              81.76   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack   89.76   
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                 80.01   
                               Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack                 NaN   
                               Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack          78.40   
                      Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                     NaN   
                                           492130008399 Yellow Tail - Merlot - 750 mL Bottle                  NaN   

                                                                                                                   \
Store                                                                                                     Store 3   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        128.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  278.80   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         204.12   
                      Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack              62.48   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack  135.72   
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                 67.36   
         


Sort by Category, then Subcategory, then UPC EAN, then Description, then Date, then Deparment:


Dollars  \
Store                                                                                                     Store 1   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            NaN   
2018-07-11 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            NaN   
2018-07-12 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            NaN   
2018-07-13 Booooze    Alcohol  Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   

                                                                                                                   \
Store                                                                                                     Store 2   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        231.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  172.00   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         168.06   
2018-07-11 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        248.80   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  201.72   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         338.10   
2018-07-12 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        279.36   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  250.25   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         410.88   
2018-07-13 Booooze    Alcohol  Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  432.74   

                                                                                                                   \
Store                                                                                                     Store 3   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        128.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  278.80   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         204.12   
2018-07-11 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        296.01   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  318.80   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         102.04   
2018-07-1


Sort by Category, then Date only:


Dollars  \
Store                                                                                                     Store 1   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            NaN   
2018-07-11 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            NaN   
2018-07-12 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            NaN   
2018-07-13 Booooze    Alcohol  Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   

                                                                                                                   \
Store                                                                                                     Store 2   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        231.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  172.00   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         168.06   
2018-07-11 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        248.80   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  201.72   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         338.10   
2018-07-12 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        279.36   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  250.25   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         410.88   
2018-07-13 Booooze    Alcohol  Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  432.74   

                                                                                                                   \
Store                                                                                                     Store 3   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        128.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  278.80   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         204.12   
2018-07-11 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        296.01   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  318.80   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         102.04   
2018-07-1

# Slicing, Filtering, and Querying

With the sorting out of the way we can begin searching our data by specific criteria. There is a fantastic guide about [understanding SettingwithCopyWarnings in Pandas](https://www.dataquest.io/blog/settingwithcopywarning/), so if you're unclear on the difference between a view and a copy, I suggest giving that a read first.

Readers who have used Pandas for slicing previously will know that .loc is generally the preferred method of referencing cells in a DataFrame. The same type of syntax exists for multiindex Dataframes:

In [48]:
# A slicing helper. Works similarly to slicing in Python (e.g. list slicing),
#                   but is inclusive of both the start and stop values.
idx = pd.IndexSlice

# View rows with a Category of Beer, but any Date, Department, Subcategory, UPC EAN, or Description
# Only looking at Dollars columns
print("View only rows in the 'Dollars' columns where the Category is 'Beer':")
display(sort3.loc[idx[:, :, 'Beer'], 'Dollars':'Dollars'].head(10))

# If we just want to look at the Store 1 sub-column
print("\nView only rows in the 'Dollars' AND 'Store 1' column where the Category is 'Beer':")
display(sort3.loc[idx[:, :, 'Beer'], idx['Dollars', 'Store 1':'Store 1']].head(10))

View only rows in the 'Dollars' columns where the Category is 'Beer':


Dollars  \
Store                                                                                                     Store 1   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack                NaN   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack     NaN   
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                   NaN   
                               Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack              137.67   
                               Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack            NaN   
2018-07-11 Booooze    Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack                NaN   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack     NaN   
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                   NaN   
                               Malts       71250000140  Colt 45 - Malt Liquor - 6 Pack                        NaN   
                               Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack            NaN   

                                                                                                                   \
Store                                                                                                     Store 2   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack              81.76   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack   89.76   
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                 80.01   
                               Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack                 NaN   
                               Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack          78.40   
2018-07-11 Booooze    Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack             104.91   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack  156.24   
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                166.74   
                               Malts       71250000140  Colt 45 - Malt Liquor - 6 Pack                     101.79   
                               Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack          56.14   

                                                                                                                   
Store                                                                                                     Store 3  
Date       Department Category Subcategory UPC EAN      Description                                                
2018-07-10 Booooze    Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack              62.48  
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack  135.72  
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                 67.36  
                               Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack              176.96  
                               Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack         119.90  
2018-07-11 Booooze    Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack              72.72  
                  


View only rows in the 'Dollars' AND 'Store 1' column where the Category is 'Beer':


Dollars
Store                                                                                                     Store 1
Date       Department Category Subcategory UPC EAN      Description                                              
2018-07-10 Booooze    Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack                NaN
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack     NaN
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                   NaN
                               Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack              137.67
                               Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack            NaN
2018-07-11 Booooze    Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack                NaN
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack     NaN
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                   NaN
                               Malts       71250000140  Colt 45 - Malt Liquor - 6 Pack                        NaN
                               Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack            NaN

Unfortunately the .loc syntax doesn't scale well to having many index levels. In order to select a values from a single level, .loc requires specifying ':' for every level that comes before. For the column names to show up for a single column you also need to use <name>:<name> syntax just to specify a single column. This can be incredibly annoying.
    
Instead, it is more common (and practical) to use df.xs (cross sections). df.xs allows us to filter our DataFrame to only those rows that match levels of our index we specify. We can choose an individual level or multiple levels easily:

In [49]:
# We provide the level value, which by default searches the first index level ('Date')
display(multi_col_lvl_df.xs('2018-07-12').head(10))

# Here we're more specific because we want to search 'Category instead'
display(multi_col_lvl_df.xs('Wine', level='Category').head(10))

# To search for rows that match all of our level value requirements, use tuples
display(multi_col_lvl_df.xs(('2018-07-10', 'Booooze', 'Wine'), level=['Date', 'Department', 'Category']).head(10))

# Note that chaining .xs works, but is significantly less efficient
display(multi_col_lvl_df.xs('2018-07-10', level='Date')
                        .xs('Booooze', level='Department')
                        .xs('Wine', level='Category')
                        .head(10))

# We can then use the column value filtering we're used to to filter rows as well
cross_section_df = multi_col_lvl_df.xs(('2018-07-10', 'Booooze'), level=['Date', 'Department'])
display(cross_section_df[cross_section_df['Dollars']['Store 1'] > 0].head(10))

Dollars  \
Store                                                                                          Store 1   
Department Category Subcategory UPC EAN      Description                                                 
Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle           NaN   
                    Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle     NaN   
                                674545000001 Don Julio - Tequila Blanco - 750 mL Bottle            NaN   
           Beer     Ales        702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack     NaN   
                                736920111112 Goose Island - Honkers Ale - 6 Pack                   NaN   
                    Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack               40.12   
                    Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack            NaN   
           Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                   60.24   
                                492130008399 Yellow Tail - Merlot - 750 mL Bottle                95.16   
                    Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle          93.94   

                                                                                                        \
Store                                                                                          Store 2   
Department Category Subcategory UPC EAN      Description                                                 
Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        279.36   
                    Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  250.25   
                                674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         410.88   
           Beer     Ales        702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack  109.20   
                                736920111112 Goose Island - Honkers Ale - 6 Pack                 37.75   
                    Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack                 NaN   
                    Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack         116.71   
           Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                     NaN   
                                492130008399 Yellow Tail - Merlot - 750 mL Bottle                  NaN   
                    Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle          83.82   

                                                                                                        \
Store                                                                                          Store 3   
Department Category Subcategory UPC EAN      Description                                                 
Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        228.08   
                    Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  455.13   
                                674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         268.56   
           Beer     Ales        702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack   56.76   
                                736920111112 Goose Island - Honkers Ale - 6 Pack                 78.30   
                    Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack               93.10   
                    Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack         109.34   
           Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                   69.29   
                                492130008399 Yellow Tail - Merlot - 750 mL Bottle                75.60   
                    Rose        85000024218  Dark Horse - S. Blan

Dollars  \
Store                                                                                      Store 1   
Date       Department Subcategory UPC EAN      Description                                           
2018-07-10 Booooze    Red         89744402585  Trapiche - Malbec - 750 mL Bottle             56.88   
                                  492130008399 Yellow Tail - Merlot - 750 mL Bottle          46.30   
                      Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle   148.61   
                      White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle     NaN   
2018-07-11 Booooze    Red         89744402585  Trapiche - Malbec - 750 mL Bottle             50.40   
                      Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle    52.24   
                      White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle     NaN   
2018-07-12 Booooze    Red         89744402585  Trapiche - Malbec - 750 mL Bottle             60.24   
                                  492130008399 Yellow Tail - Merlot - 750 mL Bottle          95.16   
                      Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle    93.94   

                                                                                                    \
Store                                                                                      Store 2   
Date       Department Subcategory UPC EAN      Description                                           
2018-07-10 Booooze    Red         89744402585  Trapiche - Malbec - 750 mL Bottle               NaN   
                                  492130008399 Yellow Tail - Merlot - 750 mL Bottle            NaN   
                      Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle   100.38   
                      White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle  293.16   
2018-07-11 Booooze    Red         89744402585  Trapiche - Malbec - 750 mL Bottle               NaN   
                      Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle   173.03   
                      White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle  237.16   
2018-07-12 Booooze    Red         89744402585  Trapiche - Malbec - 750 mL Bottle               NaN   
                                  492130008399 Yellow Tail - Merlot - 750 mL Bottle            NaN   
                      Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle    83.82   

                                                                                                    \
Store                                                                                      Store 3   
Date       Department Subcategory UPC EAN      Description                                           
2018-07-10 Booooze    Red         89744402585  Trapiche - Malbec - 750 mL Bottle             37.80   
                                  492130008399 Yellow Tail - Merlot - 750 mL Bottle          34.56   
                      Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle      NaN   
                      White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle  278.60   
2018-07-11 Booooze    Red         89744402585  Trapiche - Malbec - 750 mL Bottle             47.25   
                      Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle      NaN   
                      White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle  247.52   
2018-07-12 Booooze    Red         89744402585  Trapiche - Malbec - 750 mL Bottle             69.29   
                                  492130008399 Yellow Tail - Merlot - 750 mL Bottle          75.60   
                      Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle      NaN   

                                                                                             Units  \
Store   

Dollars  \
Store                                                                Store 1   
Subcategory UPC EAN      Description                                           
Red         89744402585  Trapiche - Malbec - 750 mL Bottle             56.88   
            492130008399 Yellow Tail - Merlot - 750 mL Bottle          46.30   
Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle   148.61   
White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle     NaN   

                                                                              \
Store                                                                Store 2   
Subcategory UPC EAN      Description                                           
Red         89744402585  Trapiche - Malbec - 750 mL Bottle               NaN   
            492130008399 Yellow Tail - Merlot - 750 mL Bottle            NaN   
Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle   100.38   
White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle  293.16   

                                                                              \
Store                                                                Store 3   
Subcategory UPC EAN      Description                                           
Red         89744402585  Trapiche - Malbec - 750 mL Bottle             37.80   
            492130008399 Yellow Tail - Merlot - 750 mL Bottle          34.56   
Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle      NaN   
White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle  278.60   

                                                                       Units  \
Store                                                                Store 1   
Subcategory UPC EAN      Description                                           
Red         89744402585  Trapiche - Malbec - 750 mL Bottle               8.0   
            492130008399 Yellow Tail - Merlot - 750 mL Bottle           10.0   
Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle     11.0   
White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle     NaN   

                                                                              \
Store                                                                Store 2   
Subcategory UPC EAN      Description                                           
Red         89744402585  Trapiche - Malbec - 750 mL Bottle               NaN   
            492130008399 Yellow Tail - Merlot - 750 mL Bottle            NaN   
Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle      7.0   
White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle    14.0   

                                                                              
Store                                                                Store 3  
Subcategory UPC EAN      Description                                          
Red         89744402585  Trapiche - Malbec - 750 mL Bottle               6.0  
            492130008399 Yellow Tail - Merlot - 750 mL Bottle            6.0  
Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle      NaN  
White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle    14.0

Dollars  \
Store                                                                Store 1   
Subcategory UPC EAN      Description                                           
Red         89744402585  Trapiche - Malbec - 750 mL Bottle             56.88   
            492130008399 Yellow Tail - Merlot - 750 mL Bottle          46.30   
Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle   148.61   
White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle     NaN   

                                                                              \
Store                                                                Store 2   
Subcategory UPC EAN      Description                                           
Red         89744402585  Trapiche - Malbec - 750 mL Bottle               NaN   
            492130008399 Yellow Tail - Merlot - 750 mL Bottle            NaN   
Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle   100.38   
White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle  293.16   

                                                                              \
Store                                                                Store 3   
Subcategory UPC EAN      Description                                           
Red         89744402585  Trapiche - Malbec - 750 mL Bottle             37.80   
            492130008399 Yellow Tail - Merlot - 750 mL Bottle          34.56   
Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle      NaN   
White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle  278.60   

                                                                       Units  \
Store                                                                Store 1   
Subcategory UPC EAN      Description                                           
Red         89744402585  Trapiche - Malbec - 750 mL Bottle               8.0   
            492130008399 Yellow Tail - Merlot - 750 mL Bottle           10.0   
Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle     11.0   
White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle     NaN   

                                                                              \
Store                                                                Store 2   
Subcategory UPC EAN      Description                                           
Red         89744402585  Trapiche - Malbec - 750 mL Bottle               NaN   
            492130008399 Yellow Tail - Merlot - 750 mL Bottle            NaN   
Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle      7.0   
White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle    14.0   

                                                                              
Store                                                                Store 3  
Subcategory UPC EAN      Description                                          
Red         89744402585  Trapiche - Malbec - 750 mL Bottle               6.0  
            492130008399 Yellow Tail - Merlot - 750 mL Bottle            6.0  
Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle      NaN  
White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle    14.0

Dollars  \
Store                                                                        Store 1   
Category Subcategory UPC EAN      Description                                          
Beer     Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack       137.67   
Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle            56.88   
                     492130008399 Yellow Tail - Merlot - 750 mL Bottle         46.30   
         Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle  148.61   

                                                                                      \
Store                                                                        Store 2   
Category Subcategory UPC EAN      Description                                          
Beer     Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack          NaN   
Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle              NaN   
                     492130008399 Yellow Tail - Merlot - 750 mL Bottle           NaN   
         Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle  100.38   

                                                                                      \
Store                                                                        Store 3   
Category Subcategory UPC EAN      Description                                          
Beer     Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack       176.96   
Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle            37.80   
                     492130008399 Yellow Tail - Merlot - 750 mL Bottle         34.56   
         Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle     NaN   

                                                                               Units  \
Store                                                                        Store 1   
Category Subcategory UPC EAN      Description                                          
Beer     Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack         13.0   
Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle              8.0   
                     492130008399 Yellow Tail - Merlot - 750 mL Bottle          10.0   
         Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle    11.0   

                                                                                      \
Store                                                                        Store 2   
Category Subcategory UPC EAN      Description                                          
Beer     Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack          NaN   
Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle              NaN   
                     492130008399 Yellow Tail - Merlot - 750 mL Bottle           NaN   
         Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle     7.0   

                                                                                      
Store                                                                        Store 3  
Category Subcategory UPC EAN      Description                                         
Beer     Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack         14.0  
Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle              6.0  
                     492130008399 Yellow Tail - Merlot - 750 mL Bottle           6.0  
         Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle     NaN

That's a whole lot nicer of syntax than the .loc method, but it's still relatively verbose and requires separating out or index and column value filtering. Multiindex DataFrames also offer a lesser known method, df.query. Query is powerful for a few reasons:
1. It can search both index levels and columns in the same query,
2. Syntactically it's nicer, since it lets use write our query as short expressions,
3. It's very efficient (depending on the chosen engine)

The documentation has excellent [notes about the method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.query.html) and a nice [usage guide](https://pandas.pydata.org/pandas-docs/stable/indexing.html#multiindex-query-syntax) that are definitely worth a read but a couple of examples using our dataset:

In [50]:
multi_col_lvl_df.query("Date == '2018-07-10' and Category == 'Wine'")

Dollars  \
Store                                                                                               Store 1   
Date       Department Category Subcategory UPC EAN      Description                                           
2018-07-10 Booooze    Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle             56.88   
                                           492130008399 Yellow Tail - Merlot - 750 mL Bottle          46.30   
                               Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle   148.61   
                               White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle     NaN   

                                                                                                             \
Store                                                                                               Store 2   
Date       Department Category Subcategory UPC EAN      Description                                           
2018-07-10 Booooze    Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle               NaN   
                                           492130008399 Yellow Tail - Merlot - 750 mL Bottle            NaN   
                               Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle   100.38   
                               White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle  293.16   

                                                                                                             \
Store                                                                                               Store 3   
Date       Department Category Subcategory UPC EAN      Description                                           
2018-07-10 Booooze    Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle             37.80   
                                           492130008399 Yellow Tail - Merlot - 750 mL Bottle          34.56   
                               Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle      NaN   
                               White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle  278.60   

                                                                                                      Units  \
Store                                                                                               Store 1   
Date       Department Category Subcategory UPC EAN      Description                                           
2018-07-10 Booooze    Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle               8.0   
                                           492130008399 Yellow Tail - Merlot - 750 mL Bottle           10.0   
                               Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle     11.0   
                               White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle     NaN   

                                                                                                             \
Store                                                                                               Store 2   
Date       Department Category Subcategory UPC EAN      Description                                           
2018-07-10 Booooze    Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle               NaN   
                                           492130008399 Yellow Tail - Merlot - 750 mL Bottle            NaN   
                               Rose        85000024218  Dark Horse - S. Blanc Rose - 750 mL Bottle      7.0   
                               White       81248501095  Curious Beasts - Chardonnay - 750 mL Bottle    14.0   

                                                                                                             
Store                                                                                               

So why doesn't everyone always use df.query? It has two big drawbacks: 
1. It doesn't allow index or column names with spaces in the name. 
2. It can't deal with multiple column levels well.

However, if you're used to working with SQL databases and/or willing to work within the above limitations, this is a powerful and fast option.

# Updating Column Values

Now that we've learned a few ways to slice, filter, and query our data, what if we want to alter values in our dataframe? We can take advantage of the .loc method to replace values in place without creating a copy or, if temporarily copying our data isn't an issue (i.e. too big for memory) we can use the df.xs method with df.update:

In [51]:
updated_df = multi_col_lvl_df.copy()
updated_df.loc[idx[:, :, 'Beer'], idx['Dollars', 'Store 1':'Store 1']] = "We changed this"
updated_df.head(10)

Dollars  \
Store                                                                                                              Store 1   
Date       Department Category Subcategory UPC EAN      Description                                                          
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle                    NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle              NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle                     NaN   
                      Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack             We changed this   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack  We changed this   
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                We changed this   
                               Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack              We changed this   
                               Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack         We changed this   
                      Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                            56.88   
                                           492130008399 Yellow Tail - Merlot - 750 mL Bottle                          46.3   

                                                                                                                   \
Store                                                                                                     Store 2   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        231.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  172.00   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         168.06   
                      Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack              81.76   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack   89.76   
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                 80.01   
                               Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack                 NaN   
                               Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack          78.40   
                      Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                     NaN   
                                           492130008399 Yellow Tail - Merlot - 750 mL Bottle                  NaN   

                                                                                                                   \
Store                                                                                                     Store 3   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        128.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  278.80   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         204.12   
                      Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack              62.48   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack  135.72   
                  

The above syntax works well for immutable values, but you'll get an error if you try the same thing for a mutable value, like a list. To my knowledge the only way to set values in a DataFrame to a mutable object is one cell at a time, so doing multiple replacements requires iterrows:

In [52]:
# # Note: If the column we're changing's type isn't already object,
# # we need to change it or the value relpacements below will error.
# updated_df['Dollars']['Store 1'] = updated_df['Dollars']['Store 1'].astype(object)

# # Loop through rows, replacing single values
# # Only necessary if the new assigned value is mutable
# # Code below currently not working when there are multiple column levels, but works with one column level
# for index, row in updated_df.loc[idx[:, :, 'Beer'], idx['Dollars', 'Store 1':'Store 1']].iterrows():
#     updated_df.at[index, idx['Dollars', 'Store 1':'Store 1']] = ["We", "changed", "this"]

# updated_df.head(3)

Alternatively, using df.xs and then df.update instead of the full .loc is a little bit more understandable in my option. For example, the following code does the same as the code two cells above (except that it requires a copy):

In [53]:
updated_df = multi_col_lvl_df.copy()

df2 = updated_df.xs('Beer', level='Category', drop_level=False).copy()  # .copy() is to avoid SettingwithCopyWarning
df2[idx['Dollars', 'Store 1']] = "We ALSO changed this"

updated_df.update(df2, join="left", overwrite=True, filter_func=None, raise_conflict=False)
updated_df.head(10)

/Users/ddl/anaconda3/envs/datascience/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: the raise_conflict=False keyword is deprecated, use errors='ignore' instead
  


Dollars  \
Store                                                                                                                   Store 1   
Date       Department Category Subcategory UPC EAN      Description                                                               
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle                         NaN   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle                   NaN   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle                          NaN   
                      Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack             We ALSO changed this   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack  We ALSO changed this   
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                We ALSO changed this   
                               Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack              We ALSO changed this   
                               Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack         We ALSO changed this   
                      Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                                 56.88   
                                           492130008399 Yellow Tail - Merlot - 750 mL Bottle                               46.3   

                                                                                                                   \
Store                                                                                                     Store 2   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        231.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  172.00   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         168.06   
                      Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack              81.76   
                                           702770082018 Omission Pale Ale - Gluten Free Pale Ale - 6 Pack   89.76   
                                           736920111112 Goose Island - Honkers Ale - 6 Pack                 80.01   
                               Lagers      702770081011 Omission - Gluten Free Lager - 6 Pack                 NaN   
                               Stouts      83783575217  Sierra Nevada Brewing Co. - Stout - 6 Pack          78.40   
                      Wine     Red         89744402585  Trapiche - Malbec - 750 mL Bottle                     NaN   
                                           492130008399 Yellow Tail - Merlot - 750 mL Bottle                  NaN   

                                                                                                                   \
Store                                                                                                     Store 3   
Date       Department Category Subcategory UPC EAN      Description                                                 
2018-07-10 Booooze    Alcohol  Liqour      80480280024  Grey Goose - Imported Vodka - 750 mL Bottle        128.30   
                               Liquor      80432400630  Glenlivet - 12 Year Scotch Whisky - 750 mL Bottle  278.80   
                                           674545000001 Don Julio - Tequila Blanco - 750 mL Bottle         204.12   
                      Beer     Ales        94922755711  Bowser Beer - Beefy Brown Ale - 6 Pack              62.48   
                                           702770082018 Omission Pale Ale -

# Display options

In [54]:
pd.set_option('display.multi_sparse', True)

# That's all, folks!

Hopefully this guide has been a gentle and practical tutorial on multiindex DataFrames in Pandas. If you find any errors, material ommissions, or topics that could just be explained more clearly, please submit leave a comment or better yet, a pull request!

To continue your learning, find a list of my top picks for Pandas resources - generally and specifically for MultiIndexes. Of particular note is the [Official list of available methods for MultiIndexes](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.MultiIndex.html), which has a full listing of all the available methods (many not listed here). **This link is specifically for Pandas version 0.22, because, for some reason, the list in future versions excludes a lot of still valuable methods!** What's more, this list provides the ONLY explanation for some methods, since many MultiIndex methods lack even the most basic of docstrings.

# Resources

**Official MultiIndex References**
 * [Official MultiIndex / Advanced Indexing Tutorial](https://pandas.pydata.org/pandas-docs/stable/advanced.html?highlight=indexslice#hierarchical-indexing-multiindex)
 * [Official list of available methods for MultiIndexes](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.MultiIndex.html)
 
**Other MultiIndex Tutorials**
 * [Official Reshaping by Stacking and Unstacking Tutorial](https://pandas.pydata.org/pandas-docs/stable/reshaping.html#reshaping-by-stacking-and-unstacking)
 * [Hierarchical indices, groupby and pandas by DataCamp](https://www.datacamp.com/community/tutorials/pandas-multi-index)
 * [Python Data Science Handbook by Jake Vanderplas](https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html#Methods-of-MultiIndex-Creation)
 * [MultiIndex slicing by Nelson](https://www.somebits.com/~nelson/pandas-multiindex-slice-demo.html)
 * [Pandas examples and cookbook by Eric Neilsen, Jr.](http://ehneilsen.net/notebook/pandasExamples/pandas_examples.html)
 
**General Pandas Tutorials**
 * [Official 10 Minutes to Pandas Tutorial](https://pandas.pydata.org/pandas-docs/stable/10min.html)
 * [Official Pandas Cheat Sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)
 * [Official Indexing and Selecting Data Tutorial](https://pandas.pydata.org/pandas-docs/stable/indexing.html)
 * [Understanding SettingwithCopyWarning in pandas by DataQuest](https://www.dataquest.io/blog/settingwithcopywarning/)
 
**Pandas Operations and Efficiency**
 * [A Beginner’s Guide to Optimizing Pandas Code for Speed by Sofia Heisler](https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6)
 * [Using pandas with large data by DataQuest](https://www.dataquest.io/blog/pandas-big-data/)
 * [Pandas Under The Hood by Jeff Tratner](http://www.jeffreytratner.com/slides/pandas-under-the-hood-pydata-seattle-2015.pdf)
 * [Pandas 2.0 Internals: Data structure changes by Wes McKinney](https://pandas-dev.github.io/pandas2/internal-architecture.html)
